In [8]:
#Imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog

## Import data, shuffle


In [9]:
def read_rgb(img_file):
    """
    To be consistent with reading images, I'll use cv2, which reads images
    as BGR with scale 0 to 255.
    
    Since matplotlib.image scales 0 to 1 for png but 0 to 255 for jpg, as RGB,
    I'll chooose cv2 to read the image, then convert BGR to RGB
    I will keep the scale as 0 to 255, because if I change color spaces,
    the scale will be 0 to 255 again.
    
    I'll use StandardScaler() to standardize features later in the pipeline.
    """
    img = cv2.imread(img_file)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [30]:
def convert_color(img, color_space='RGB'):
    """
    img: an RGB image
    color_space: HSV, LUV, HLS, YUV, YCrCb
    returns: a new image converted to the chosen color space, else a copy of the original image
    """
    if color_space == 'HSV':
        img_out = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    elif color_space == 'LUV':
        img_out = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    elif color_space == 'HLS':
        img_out = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    elif color_space == 'YUV':
        img_out = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    elif color_space == 'YCrCb':
        img_out = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        img_out = np.copy(img)
        
    return img_out

In [11]:
def bin_spatial(img, size=(32, 32)):
    """
    compute binned color features  
    Use cv2.resize().ravel() to create the feature vector
    ravel flattens the n-dim array to a 1D array
    """
    features = cv2.resize(img, size).ravel() 
    return features

In [29]:
def color_hist(img, nbins=32, bins_range=(0, 256)):
    """
    compute color histogram features
    numpy.histogram returns both the histogram counts and the bins as array index 0 and 1
    Make the list of arrays into an array, 
    then flatten as if we're concatenating each histogram one after the other
    returns a numpy array of histogram values for all channels
    """
    hists = []
    for channel in range(img.shape[2]):
        hist, bind_edges = np.histogram(img[:,:,channel], bins=nbins, range=bins_range)
        hists.append(hist)
    hist_features = np.array(hists).ravel() 
    return hist_features

In [16]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True, hog_channel='All'):
    """
    return HOG features and visualization
    img: image from which to extract histogram of gradients,
        will have 3 channels.
    orient: number of gradient orientations
    pix_per_cell: size of cell in pixels
    cell_per_block: size of block (in cells)
    vis: if True, returns an image representing the found histogram of gradients
    feature_vec: if True, flattens result into 1d array using numpy.ravel()
    hog_channel: 'All' uses all channels, else 0 to 3 representing the channel to use
    """
    if hog_channel == 'ALL': #use all channels
        hog_features = []
        hog_images = []
        for channel in range(img.shape[2]):
            features, hog_image = hog(img[:,:,channel],
                                      orientations=orient,
                                      pixels_per_cell=(pix_per_cell, pix_per_cell),
                                      cells_per_block=(cell_per_block, cell_per_block),
                                      transform_sqrt=True, 
                                      visualise=vis,
                                      feature_vector=feature_vec)
            hog_features.append(features)
            hog_images.append(hog_image)
        hog_features = np.ravel(hog_features)
    else: #use chosen channel
        hog_features, hog_images = hog(img[:,:,hog_channel],
                                  orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=True, 
                                  visualise=vis,
                                  feature_vector=feature_vec)
    if vis == True:
        return hog_features, hog_images
    else:      
        return hog_features

In [17]:
def extract_features(paths, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256)):
    """
    paths: list of file paths for images (either vehicle or non-vehicle)
    color_space: HSV, LUV, HLS, YUV, YCrCb, otherwise keeps as RGB.
    
    """
    features = []
    for file in paths:
        image = read_rgb(file)
        feature_image = convert_color(image, color_space)
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        hog_features = get_hog_features(feature_image, orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True)
        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    return features

In [ ]:
#def get_data():
cars_paths = glob.glob('./vehicles/*/*.png') #8792 
other_paths = glob.glob('./non-vehicles/*/*.png') #8968
